# Bayes starting point search

The idea here is to use an approximation of Bayesian model comparison to identify the starting point `x1` where the linar part of a given time series transitions toa sigmoidal part

In [ ]:
%matplotlib notebook
import matplotlib.pyplot as plt

import pandas as pd
import numpy as np

import random
random.seed(123) # For repeatability

## Generate synthetic data

To be able to assess the quality of our model we first generate a large number of synthetic traces plus the correct `x1`.

In [ ]:
N = 10  # number of traces
data = pd.DataFrame()

for i in range(N):
    S = random.randint(100, 200)  # number of samples
    y0 = random.uniform(1, 20)  # offset
    s = random.uniform(-0.1, 0.2)  # slope of linear part
    std = random.uniform(1, 15)  # noise
    x12 = random.randint(50, S - 50)  # halfway-point of sigmoid
    xSigWidth = random.uniform(0.5, 30)  # width scaling factor of sigmoid
    ySig = random.uniform(20, 300)  # height of sigmoid

    x = np.arange(start=0, stop=S)
    y = (
        y0
        + s * x
        + ySig * np.tanh((x - x12) / xSigWidth)
        + np.random.normal(scale=std, size=S)
    )
    data = data.append({"x": x, "y": y, "S": S, "x12": x12, "xSigWidth": xSigWidth}, ignore_index=True)

In [ ]:
data.head()

In [ ]:
fig = plt.figure(figsize=(9.5, 4))

for i in range(min(N, 4)):
    p = fig.add_subplot(1, 4, i + 1)
    #     p.set(
    #         xlabel="sample",
    #         ylabel="Vertical displacement / mm",
    #         title="Theoretical displacement / mm",
    #     )
    p.plot(data.loc[i].x, data.loc[i].y)